# 1. Configurar ambiente

In [ ]:
import pandas as pd
import re
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
!pip install jellyfish
import jellyfish

#2. Tratamiento de datos

In [ ]:
def tratamiento_texto(texto):
  trans = str.maketrans('áéíóú','aeiou')
  texto = texto.lower()
  #print('lower: ',texto)
  texto = texto.translate(trans)
  #print('acentuación: ',texto)
  texto = re.sub(r"[^\w\s]", '', texto)
  #print('símbolos: ',texto)
  texto = " ".join(texto.split())
  return texto

# 3. Cargar bases de conocimiento

In [ ]:
#Importando bases de dialogo fluído
txt_folder_path = '/content'
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".txt")]
lista_dialogos, lista_dialogos_respuesta, lista_tipo_dialogo = [],[],[]
for idx in range(len(lista_documentos)):
  f=open(txt_folder_path+'/'+lista_documentos[idx], 'r', encoding='utf-8', errors='ignore')
  flag,posicion = True,0
  for line in f.read().split('\n'):
    if flag:
      line = tratamiento_texto(line)
      lista_dialogos.append(line)
      lista_tipo_dialogo.append(lista_documentos[idx][:2])
    else:
      lista_dialogos_respuesta.append(line)
      posicion+=1
    flag=not flag

#Creando Dataframe de diálogos
datos = {'dialogo':lista_dialogos,'respuesta':lista_dialogos_respuesta,'tipo':lista_tipo_dialogo,'interseccion':0,'similarity':0,'jaro_winkler':0,'probabilidad':0}
df_dialogo = pd.DataFrame(datos)
df_dialogo = df_dialogo.drop_duplicates(keep='first')
df_dialogo.reset_index(drop=True, inplace=True)

# 4. Buscar respuesta del Chatbot

In [ ]:
vectorizer = TfidfVectorizer()
dialogos_numero = vectorizer.fit_transform(df_dialogo['dialogo'])

#Función para verificar si el usuário inició un diálogo
def dialogo(user_response):
  df = df_dialogo.copy()
  respuesta_numero = vectorizer.transform([user_response])
  for idx,row in df.iterrows():
    df.at[idx,'interseccion'] = len(set(user_response.split()) & set(row['dialogo'].split()))/len(user_response.split())
    df.at[idx,'similarity'] = cosine_similarity(dialogos_numero[idx], respuesta_numero)[0][0]
    df.at[idx,'jaro_winkler'] = jellyfish.jaro_winkler(user_response,row['dialogo'])
    df.at[idx,'probabilidad'] = max(df.at[idx,'interseccion'],df.at[idx,'similarity'],df.at[idx,'jaro_winkler'])
  df.sort_values(by=['probabilidad','jaro_winkler'], inplace=True, ascending=False)
  return df.head(5)

# 5. Ejecutar Chatbot

In [ ]:
pregunta='sabes hacer que?'
user_response = tratamiento_texto(pregunta)
print(user_response)
respuesta = dialogo(user_response)
respuesta
#print(respuesta['respuesta'].head(1).values[0])

sabes hacer que


,dialogo,respuesta,tipo,interseccion,similarity,jaro_winkler,probabilidad
148,que sabes hacer,"Puedo responder muchas preguntas, sé todo sobr...",ID,1.000000,1.000000,0.757265,1.000000
145,puedes hacer mas de una cosa a la vez,"Claro, estoy diseñado para manejar múltiples t...",ID,0.333333,0.203335,0.688057,0.688057
168,buenas tardes como has estado,"He estado bien, gracias. ¿Y tú?",SA,0.000000,0.000000,0.686316,0.686316
138,puedes hacer magia bro,"Jajaja, no exactamente. Pero puedo hacer mucha...",ID,0.333333,0.265465,0.683838,0.683838
169,buenas noches como has estado,"He estado bien, gracias. ¿Y tú?",SA,0.000000,0.000000,0.669083,0.669083


#6. Validaciones

In [ ]:
#Tratamiento de texto
texto1 = '¡Hola! ¿Quién eres?'
print(tratamiento_texto(texto1))

In [ ]:
#Base de dialogos
df_dialogo

In [ ]:
#Intersección
texto1 = 'quien eres tu'
texto2 = 'hola quien eres'
print(len(set(texto1.split()) & set(texto2.split()))/len(texto1.split()))

0.6666666666666666


In [ ]:
#Similaridad coseno
texto1 = 'quien eres tu'
texto2 = 'hola quien eres'
texto1 = vectorizer.transform([texto1])
texto2 = vectorizer.transform([texto2])
#print(texto1)
print(cosine_similarity(texto1, texto2)[0][0])

0.8106505819075088


In [ ]:
#Jaro-Winkler
texto1 = 'quien eres tu'
texto2 = 'hola quien eres'
print(jellyfish.jaro_winkler(texto1,texto2))

0.7083139083139084
